In [1]:
import os 
import json 
import torch
import numpy as np 
from PIL import Image
from evaluate import load
import matplotlib.pyplot as plt
from datasets import load_dataset 
from tqdm.notebook import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoProcessor, AutoModelForCausalLM,  ViTImageProcessor, AutoTokenizer

In [2]:
model_configs = {
    "blip": {
        "saved_checkpoint": "checkpoints/models/blip/",
        "processor": BlipProcessor, 
        "pretrained_path": "Salesforce/blip-image-captioning-large", 
        "inference_model": BlipForConditionalGeneration
    }, 
    "git": {
        "saved_checkpoint": "checkpoints/models/git/",
        "processor": AutoProcessor, 
        "pretrained_path": "microsoft/git-base", 
        "inference_model": AutoModelForCausalLM
    },
    "vit": {
        "saved_checkpoint": "checkpoints/models/vit/",
        "processor": [ViTImageProcessor, AutoTokenizer], 
        "pretrained_path": ["nlpconnect/vit-gpt2-image-captioning", "microsoft/git-base"], 
        "inference_model": AutoModelForCausalLM
    }
}

In [3]:
DATA_PATHS = {
    "annotated_set_dir": "prompts/",
    "inference_set_dir": "complete_image_set/inference/"
}

In [4]:
def load_models(model_configs):
    models = {key: {} for key in model_configs.keys()}

    for model_name in model_configs.keys(): 
        ckpt, processor, pretrained_path, inf_model = model_configs[model_name].values()

        models[model_name]["model"] = inf_model.from_pretrained(ckpt) 
        models[model_name]["processor"] = [
            processor[0].from_pretrained(pretrained_path[0], kwargs="padding"), 
            processor[1].from_pretrained(pretrained_path[1], kwargs="padding")
        ] if model_name == "vit" else processor.from_pretrained(pretrained_path, kwargs="padding")
        
    return models

In [5]:
models = load_models(model_configs)

In [6]:
# Trials setup
inference_configs = [
    {
        "temperature": 0.8,
        "top_k": 50, 
        "do_sample": True
    }, 
    {
        "temperature": 0.5,
        "top_k": 100, 
        "do_sample": True
    },
    {
        "temperature": 0.7, 
        "top_k": 150, 
        "do_sample": True, 
        "top_p": 0.9
    }, 
    {
        "temperature": 0.3,
        "top_k": 10, 
        "do_sample": True, 
        "top_p": 0.3
    }, 
    {
        "temperature": 0.8, 
        "top_k": 20, 
        "do_sample": True, 
        "num_beams": 5
    }, 
    {
        "top_k": 20, 
        "do_sample": False, 
        "num_beams": 5,
        "num_beam_groups": 5, 
        "repetition_penalty": 1.2, 
        "diversity_penalty": 2.0
    }, 
    {
        "top_k": 20, 
        "do_sample": False, 
        "num_beams": 5,
        "num_beam_groups": 5,
        "repetition_penalty": 1.5, 
        "diversity_penalty": 1.0
    }, 
    {
        "top_k": 20, 
        "do_sample": False, 
        "num_beams": 5,
        "num_beam_groups": 5,
        "repetition_penalty": 1.8, 
        "diversity_penalty": 1.5
    }, 
]

In [12]:
# Testing configs block 
image = Image.open(DATA_PATHS["inference_set_dir"] + "404.png")
model, processor = models["git"].values()
pixel_values = processor(images=image, return_tensors="pt").pixel_values

for trial in inference_configs:   
    generated_ids = model.generate(pixel_values=pixel_values, max_length=200, num_return_sequences=5, **trial)
    processor.batch_decode(generated_ids, skip_special_tokens=True)

In [46]:
def load_config_comparison(model_name, file_name): 
    image_path = DATA_PATHS["inference_set_dir"] + file_name
    inference_dict = {} 
    image_processor = None 
    model, processor = models[model_name].values()
    image = Image.open(image_path)

    if model_name == "vit": 
        image_processor, processor = processor
    
    for index, trial in enumerate(inference_configs):                     
        pixel_values = image_processor(images=image, return_tensors="pt").pixel_values if model_name == "vit" else processor(images=image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values=pixel_values, max_length=200, num_return_sequences=5, **trial)

        inference_dict[index] = processor.batch_decode(generated_ids, skip_special_tokens=True)

    
    return inference_dict

In [47]:
def print_inferences(model_name, file_name): 
    inference = load_config_comparison(model_name, file_name) 
    
    for trial_number, captions in inference.items(): 
        print("Trial #: ", trial_number)  
    
        for caption in captions: 
            print(caption) 
    
        print()

#### GIT inference config tuning 

In [13]:
print_inferences("git", "404.png") 

Trial #:  0
design an elegant about me page with a white background, add subtle elements to the page design
design an about us page with vertical stacked layout
about us page design with centered content
design an about me page with simple text and a white background, splashes of color, shapes arranged overlaid in a 3 column shape with 2 columns
design an attractive, modern landing page design for a furniture brand with an even text - image split layout

Trial #:  1
design an elegant about me page design with centered text
design a simple about me page with minimal text and white color scheme
design a simple yet elegant about me page with minimal text
design a simple about me page with a white background, and black font color, with a single image in a straight border
design a simple about me page with a white background, and black font color, with a single image placed vertically stacked layout

Trial #:  2
design a simple about us page with minimal text
design a simple about us page w

In [14]:
print_inferences("git", "about_test.jpg") 

Trial #:  0
design an about me page with a twist, dynamic, with a split as the image of another section arranged in 2 sections
design a simple about me page with a minimalistic and elegant look with white typography against a background, with a 2 column layout, a vertically stacked layout for content
design a minimalistic and elegant about me page design with an even image of the product at the right and the image of the page, with a horizontal card section in a horizontal card layout
about us page design for an animal info application with a white background, and a profile image arranged in eight sections
design a cool about me page with a single image, with a white background, add an actual text to the right and another section in the left side

Trial #:  1
landing page design for finance app with a white background, and red color scheme
design a sleek about us page with minimal text
design a simple about me page with a white background, with a colorful animated illustration of space

In [15]:
print_inferences("git", "about_us_test.png")

Trial #:  0
design an about us page with a dark color scheme, with centered text overlaid, single section, and split into 2 sections
design an about me page with an even text - image split layout for a web design with centered content
design a simple about me page with a simple yet elegant look
landing page design for hotel brand with card layout
design a full screen hero section for an empty food plate with a white background and black font color, with a centered content in a horizontal grid

Trial #:  1
design a simple about me page with a single image, with a curved background, with a white section, and black font color, with a single image column and section content left side by side
design a simple and neat about me page with a white background, add a colorful animated animated image to add focus and circle shaped background to add links to other pages in a horizontal card layout
design a simple and neat about me page design for a project with a vertically stacked layout
landing p

#### BLIP inference config tuning 

In [ ]:
Image.open(DATA_PATHS["inference_set_dir"] + "404.png")

In [16]:
print_inferences("blip", "404.png") 

Trial #:  0
simple white background, alternating text for the heading of the page
simple text art based 404 page design with a white background and a circular logo followed by the section content
create a captivating about me page design with white and grey color scheme, huge image section, and black typography
design a simple white and grey color schemed 404 section content that is centered on the page with a circular layout that gives off an animated twist and artistic twist
create a simple 404 page design with a white theme, black font and a circle shaped background with an abstract image

Trial #:  1
simple white background, centered text, and a circular shape, gives off a modern vibe for the page design
create an elegant 404 page design with a white background, giant typography, and a circular section title that is centered with a black typography and a circular image
create a simple text graphic based 404 page design with a white background and a circular graphic arranged into a 

In [17]:
print_inferences("blip", "about_test.jpg") 

Trial #:  0
simple about me page, about us section, about us page, about us page, about us page design, about us page layout, about us page design, about us page page
develop an elegant landing page design for an educational application with a grey and white theme, a black and white section, and a yellow section that has images of people working with laptop in a laptop and a laptop as background
create an elegant landing page ui for a furniture brand with a black and white section backgrounds, rectangular, with 2 sections that are too stacked and add for page page
create an elegant about us page design with an even text - navigating section, white background, and black and white color scheme
create an elegant landing page for a card game with a 2 - panel about me page design

Trial #:  1
create an elegant landing page ui for an interior design brand with an elegant grey and white theme
create an elegant landing page design for a furniture brand with an elegant interior color palette wi

In [18]:
print_inferences("blip", "about_us_test.png")

Trial #:  0
medical application landing page
medical application landing page design with a clean white background, 2 sections with medical themed illustrations and a blue rectangular background
medical application landing page
sports academy, sports section, sports academy, sports landing page, sports sports page, sports academy, sports blog, sports page, sports academy site
3 sections in a white background themed catalog page

Trial #:  1
medical application landing page with 2 sections
medical application landing page with card layout
medical application landing page
medical application landing page
medical application landing page design with white background and colorful illustrations

Trial #:  2
medical application landing page design with 2 sections with an image carousel, white background, and colorful illustrations
white, blue color palette, education theme, education page, educational section, school theme, school themed, sports themed page, sports themed page, sports themed

#### VIT inference config tuning 

In [48]:
print_inferences("vit", "404.png") 

Trial #:  0
404 page design with a na stacked
design a section for a web page with a 404 section, 3d image, illustration with a single section for a na type
404 page design with a dinosaur with a cartoon face and a white profile layout, with a black font and a single image of a dinosaur in a purple font
design a 404 page with just a section of text that is on a column
about me page design with a white background, black text and a dark image of a person wearing a headband and holding a white object

Trial #:  1
design a 404 page with a dark color palette, white text and a white background, with a black image and a white image that has a minimal image and a white image of a purple and
landing page design for a 404 page design with a single section of text
dark landing page design for a furniture brand with a minimalistic illustration of a dinosaur image and a white background
design a minimalistic about us page with a dark theme with a white background, a white typography with a dark ima

In [49]:
print_inferences("vit", "about_test.jpg") 

Trial #:  0
landing page design with an image of a yellow and white theme
about us page design with minimal section content
about us page design with yellow background, minimalistic typography with a yellow color palette, black and white
yellow and black color scheme with white text and a black background, 2 columns
yellow ui ui for a website landing page design with right - curve shapes and white text overlay

Trial #:  1
yellow color scheme with white background, black and white color scheme, with a black theme, and a white text layout
yellow and black color scheme, with a yellow background, a white text and a black image, the page layout has a grid layout for a yellow and black color scheme, and a white image
a simple about us page design with yellow and black color scheme
about us page design with yellow and black color scheme, white background, and a simple yellow section
yellow and black color scheme with a yellow background, white text and a black typography

Trial #:  2
design 

In [50]:
print_inferences("vit", "about_us_test.png")

Trial #:  0
themed landing page design for a furniture brand in a 2 column layout with a white and black color palette and 2 column layouts
design a simple about me page design for a food based brand
simple ui design for a restaurant with white and dark blue color scheme
design a simple about me page with simple designs
design a 404 section with a green background, a white background, two sections for images, a white image and an image of a green image, a white background, a brown circle with a

Trial #:  1
design a simple about us page with a nab theme and 2 sections
design a simple about me page design for a website
about us page design with a green background, white and blue color scheme
design a simple about me page design with a simple typography based image in a grid with a white background, with a purple and white color scheme
about us page design with 2 sections

Trial #:  2
about us page design with 2 columns, 2 columns, and a section with 4 columns for a simple text layout
ab